# Import library

In [57]:
import tensorflow as tf
import requests
import json
import base64
import requests
from pprint import PrettyPrinter
import requests


In [58]:
pp = PrettyPrinter()
pp.pprint(requests.get("https://bankmarket-prediction-production.up.railway.app/v1/models/bank-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1695183648'}]}


# Set data test to predict

In [59]:
data_test= [
    {
        "age": 59,
        "job": "admin",
        "marital": "married",
        "education": "secondary",
        "default": "no",
        "balance": 2343,
        "housing": "yes",
        "loan": "no",
        "contact": "unknown",
        "day": 5,
        "month": "may",
        "duration": 1042,
        "campaign": 1,
        "pdays": -1,
        "previous": 0,
        "poutcome": "unknown",
    },
    {
        "age": 56,
        "job": "admin",
        "marital": "married",
        "education": "secondary",
        "default": "no",
        "balance": 45,
        "housing": "no",
        "loan": "no",
        "contact": "unknown",
        "day": 5,
        "month": "may",
        "duration": 1467,
        "campaign": 1,
        "pdays": -1,
        "previous": 0,
        "poutcome": "unknown",
    },
    {
        "age": 41,
        "job": "technician",
        "marital": "married",
        "education": "secondary",
        "default": "no",
        "balance": 1270,
        "housing": "yes",
        "loan": "no",
        "contact": "unknown",
        "day": 5,
        "month": "may",
        "duration": 1389,
        "campaign": 1,
        "pdays": -1,
        "previous": 0,
        "poutcome": "unknown",
    },
    {
    "age": 33,
    "job": "blue-collar",
    "marital": "single",
    "education": "primary",
    "default": "no",
    "balance": 1,
    "housing": "yes",
    "loan": "no",
    "contact": "cellular",
    "day": 20,
    "month": "apr",
    "duration": 257,
    "campaign": 1,
    "pdays": -1,
    "previous": 0,
    "poutcome": "unknown",
},
{
    "age": 43,
    "job": "technician",
    "marital": "married",
    "education": "secondary",
    "default": "no",
    "balance": 0,
    "housing": "no",
    "loan": "yes",
    "contact": "cellular",
    "day": 8,
    "month": "may",
    "duration": 9,
    "campaign": 2,
    "pdays": 172,
    "previous": 5,
    "poutcome": "failure",
}

]


# Prepocessing Data test

In [85]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )
    
def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [86]:
def prepare_json_for_serving(inputs: dict):
    data = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, int):
            data[keys] = int_feature(values)
        elif isinstance(values, str):
            data[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=data)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [87]:
def predict_depo(inputs):
    json_data = prepare_json_for_serving(inputs)
    
    endpoint = "https://bankmarket-prediction-production.up.railway.app/v1/models/bank-model:predict"
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()

    prediction = response.json()["predictions"][0][0]
    
    if prediction < 0.6:
        return "Deposit"
    else:
        return "No deposit"

# Make prediction

In [84]:
for text in data_test:
    result = predict_depo(text)
    print(result)

Deposit
Deposit
No deposit
Deposit
Deposit
